In [ ]:
# CRIA O AMBIENTE VIRTUAL

conda create -n PEFT_LoRA python=3.11
conda activate PEFT_LoRA

In [ ]:
# INSTALA AS DEPENDENCIAS
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip install transformers==4.41.2 datasets==2.19.1 accelerate==0.30.1 tokenizers==0.19.1 peft==0.11.1 bitsandbytes==0.43.1 evaluate==0.4.2 scikit-learn==1.5.0

pip install ipykernel matplotlib
python -m ipykernel install --user --name=PEFT_LoRA --display-name="PEFT_LoRA"

In [68]:
import tqdm as notebook_tqdm
import json
import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer, SFTConfig

In [69]:
# CONFIGURA OS PARÂMETROS E CARREGA O DATASET

MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_FILE = "dataset_instrucoes_juridico.jsonl"
OUTPUT_DIR = "./modelo_juridico_adaptado_LoRA"

# Carrega o dataset a partir do arquivo JSONL
full_dataset = load_dataset('json', data_files={'train': DATASET_FILE}, split='train')
train_test_split = full_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
test_validation_split = train_test_split['test'].train_test_split(test_size=0.5, shuffle=True, seed=42)

final_datasets = DatasetDict({
    'train': train_test_split['train'],
    'validation': test_validation_split['train'],
    'test': test_validation_split['test']
})

def formatar_prompt_juridico(ex):
    messages = [
        {"role":"system","content":"Você é um assistente jurídico."},
        {"role":"user","content": ex["titulo"]},
        {"role":"assistant","content": ex["conteudo"]},
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    ) + tokenizer.eos_token


print("Dataset carregado e dividido com sucesso:")
print(final_datasets)

Dataset carregado e dividido com sucesso:
DatasetDict({
    train: Dataset({
        features: ['titulo', 'conteudo', 'link'],
        num_rows: 1276
    })
    validation: Dataset({
        features: ['titulo', 'conteudo', 'link'],
        num_rows: 160
    })
    test: Dataset({
        features: ['titulo', 'conteudo', 'link'],
        num_rows: 160
    })
})


In [73]:
# CARREGAR MODELO E TOKENIZADOR

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Define o pad_token se não existir
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # Ajuste para o modelo Qwen: o pad_token_id deve ser o eos_token_id
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Carrega o Modelo em bfloat16 para economizar memória
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,  # Carrega em 16-bit (essencial para GPUs de consumidor)
    device_map="auto",           # Mapeia o modelo para a GPU automaticamente
    trust_remote_code=True
)
model.config.use_cache = False
model.config.pad_token_id = tokenizer.pad_token_id

print("Modelo e Tokenizador carregados com sucesso em 16-bit (bfloat16).")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Modelo e Tokenizador carregados com sucesso em 16-bit (bfloat16).


In [74]:
# CONFIGURAÇÃO DO LoRA

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Imprimir um resumo do modelo para confirmar a pequena porcentagem de parâmetros que serão treinados.
model.print_trainable_parameters()

trainable params: 8,798,208 || all params: 502,830,976 || trainable%: 1.7497


In [75]:
# CONFIGURAR E EXECUTAR O TREINAMENTO COM SFTTrainer

# Argumentos de Treinamento
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    warmup_ratio=0.03,
    num_train_epochs=5,
    logging_steps=10,
    bf16=True,
    save_strategy="epoch",
    eval_strategy="epoch",  # Avalia o modelo no final de cada época
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,  # Carrega o melhor modelo (baseado na perda de validação) no final
    overwrite_output_dir=True,
    weight_decay=0.02,
)

# Instanciar o SFTTrainer
trainer = SFTTrainer(
    model=model,
    peft_config=lora_config,
    formatting_func=formatar_prompt_juridico,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=final_datasets['train'],
    eval_dataset=final_datasets['validation'],
    packing=True,
    max_seq_length=1024
)

# Inicia o treinamento
trainer.train()

# Salva o modelo treinado (apenas os adaptadores)
print(f"Treinamento concluído. Salvando os adaptadores em: {OUTPUT_DIR}")
trainer.save_model()

/home/ricardo/.conda/envs/PEFT_LoRA/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ricardo/.conda/envs/PEFT_LoRA/lib/python3.11/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/ricardo/.conda/envs/PEFT_LoRA/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:181: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ricardo/.conda/envs/PEFT_LoRA/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `

Epoch,Training Loss,Validation Loss
0,2.224600,2.143196
1,2.023100,2.057740
2,1.968200,2.025370
4,1.887300,2.013012


/home/ricardo/.conda/envs/PEFT_LoRA/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ricardo/.conda/envs/PEFT_LoRA/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ricardo/.conda/envs/PEFT_LoRA/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ricardo/.conda/envs/PEFT_LoRA/lib/python3.11/site-packages/huggingface

Treinamento concluído. Salvando os adaptadores em: ./modelo_juridico_adaptado_LoRA


/home/ricardo/.conda/envs/PEFT_LoRA/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# GRÁFICO DE PERDA

import matplotlib.pyplot as plt
import pandas as pd

log_history = trainer.state.log_history
df = pd.DataFrame(log_history)

train_logs = df[df['loss'].notna()].copy()
eval_logs = df[df['eval_loss'].notna()].copy()

train_logs['epoch'] = train_logs['epoch'].round()
eval_logs['epoch'] = eval_logs['epoch'].round()

plt.figure(figsize=(12, 7))
plt.plot(
    eval_logs["epoch"],
    eval_logs["eval_loss"],
    label="Validation Loss",
    marker='o', # marcador de época
    linestyle='-',
    color='red'
)

# Curva de perda de treinamento
plt.plot(
    train_logs["epoch"],
    train_logs["loss"],
    label="Training Loss",
    marker='s', # marcador quadrado para diferenciar
    linestyle='--',
    color='blue'
)

plt.title("Curva de Perda (Loss) Durante o Treinamento", fontsize=16)
plt.xlabel("Época (Epoch)", fontsize=12)
plt.ylabel("Perda (Loss)", fontsize=12)
plt.legend()

# Add Grade
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [76]:
# VERIFICAÇÃO E DEMONSTRAÇÃO


# --- Prompt de Teste ---
prompt = "Explique em poucas palavras o que se entende pelo princípio da justiça cosmopolita em relação aos tratados internacionais?"
formatted_prompt = f"### Instrução:\n{prompt}\n\n### Resposta:\n"


# Carregar o modelo base original para comparação
base_model = model.get_base_model() 

# Carregar o modelo com os adaptadores PEFT
peft_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

# Gerar Respostas
device = "cuda:0"
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)


print("--- [TESTE 1] Resposta do Modelo Original (sem fine-tuning) ---")
with torch.no_grad():
    outputs = base_model.generate(**inputs, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print("\n" + "="*50 + "\n")

print("--- [TESTE 2] Resposta do Modelo com Adapters (pós fine-tuning) ---")
with torch.no_grad():
    outputs = peft_model.generate(**inputs, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

--- [TESTE 1] Resposta do Modelo Original (sem fine-tuning) ---
### Instrução:
Explique em poucas palavras o que se entende pelo princípio da justiça cosmopolita em relação aos tratados internacionais?

### Resposta:
O princípio da “justiça cosmopolita” é um conceito adotado pela ONU para definir a definição do direito internacional. É aquilo que, de acordo com o princípio, pode ser exercido por qualquer pessoa ou instituição no mundo todo, não importando a nacionalidade, a religião ou a crença. Assim, não se admite que uma determinada norma seja aplicada somente em determinadas regiões ou países. Ajustar-se-ia ao princípio da independência jurídica, que prevê que os Estados devem ser independentes e não dependentes de outros.  Material extraído da obra Revisaço Direito Penal


--- [TESTE 2] Resposta do Modelo com Adapters (pós fine-tuning) ---
### Instrução:
Explique em poucas palavras o que se entende pelo princípio da justiça cosmopolita em relação aos tratados internacionais?

### 